### This file (cfb_ref_cleaning.ipynb) will:
 + take in the list of all quarterback college_stats from cfb_ref_sourcing_output.pkl
 + clean data 
 + prepare for export/merge with NFL football data for each player

In [1]:
import pandas as pd


In [2]:
# Read the PKls
import pickle
# combined_df     (DataFrame)
pkl_path1 = "../sourcing_artifacts/cfb_ref_sourcing_output.pkl"
with open(pkl_path1, 'rb') as f:
    df = pickle.load(f)

display(f'rows : {len(df)}; cols: {len(df.columns)}')

'rows : 535; cols: 20'

In [3]:
# Inspect data

display(df.iloc[:, :15].head())
display(df.iloc[:, 15:27].head())
# display(df.iloc[:, 27:45].head())
# display(df.iloc[:, 50:].head())
display(df.info())
display(df.columns)
display(len(df))

,player,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate
0,Greg Cook,29,332,655,50.7,4906,34,5.2,35,5.3,7.5,6.12,14.8,169.2,120.0
1,Marty Domres,27,368,702,52.4,4492,24,3.4,38,5.4,6.4,4.65,12.2,166.4,106.6
2,Terry Hanratty,29,304,550,55.3,4152,27,4.9,34,6.2,7.5,5.75,13.7,143.2,122.5
3,Bobby Douglass,30,183,379,48.3,2817,20,5.3,20,5.3,7.4,6.11,15.4,93.9,117.6
4,Al Woodall,20,118,221,53.4,1501,4,1.8,13,5.9,6.8,4.51,12.7,75.1,104.7


,blank,awards,school,draft,pro_stats
0,,,Cincinnati,None,https://www.pro-football-reference.com/players...
1,,,Columbia,None,https://www.pro-football-reference.com/players...
2,,"H-8, H-10, H-3, AA",Notre Dame,"Draft: 2nd round, 30th overall of the 1969 NFL...",https://www.pro-football-reference.com/players...
3,,H-7,Kansas,"Draft: 2nd round, 41st overall of the 1969 NFL...",https://www.pro-football-reference.com/players...
4,,,Duke,"Draft: 2nd round, 52nd overall of the 1969 NFL...",https://www.pro-football-reference.com/players...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     535 non-null    object
 1   G          535 non-null    object
 2   Cmp        535 non-null    object
 3   Att        535 non-null    object
 4   Cmp%       535 non-null    object
 5   Yds        535 non-null    object
 6   TD         535 non-null    object
 7   TD%        535 non-null    object
 8   Int        535 non-null    object
 9   Int%       535 non-null    object
 10  Y/A        535 non-null    object
 11  AY/A       535 non-null    object
 12  Y/C        535 non-null    object
 13  Y/G        535 non-null    object
 14  Rate       535 non-null    object
 15  blank      533 non-null    object
 16  awards     535 non-null    object
 17  school     487 non-null    object
 18  draft      491 non-null    object
 19  pro_stats  528 non-null    object
dtypes: object(20)
memory usage: 83.7

None

Index(['player', 'G', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%',
       'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'blank', 'awards', 'school',
       'draft', 'pro_stats'],
      dtype='object')

535

In [4]:
pd.set_option('display.max_colwidth', None)
display(df[['player','draft']]).head()

,player,draft
0,Greg Cook,None
1,Marty Domres,None
2,Terry Hanratty,"Draft: 2nd round, 30th overall of the 1969 NFL draft by the Pittsburgh Steelers"
3,Bobby Douglass,"Draft: 2nd round, 41st overall of the 1969 NFL draft by the Chicago Bears"
4,Al Woodall,"Draft: 2nd round, 52nd overall of the 1969 NFL draft by the New York Jets"
...,...,...
530,Jake Luton,"Draft: 6th round, 189th overall of the 2020 NFL draft by the Jacksonville Jaguars"
531,Cole McDonald,"Draft: 7th round, 224th overall of the 2020 NFL draft by the Tennessee Titans"
532,Ben DiNucci,"Draft: 7th round, 231st overall of the 2020 NFL draft by the Dallas Cowboys"
533,Tommy Stevens,"Draft: 7th round, 240th overall of the 2020 NFL draft by the New Orleans Saints"


AttributeError: 'NoneType' object has no attribute 'head'

In [5]:
import pandas as pd
import ast
import re

def split_draft_col(draft):
    if pd.isna(draft):
        return pd.Series([None, None, None, None], index=['draft_rd', 'draft_overall', 'draft_yr', 'draft_team'])
    
    try:
        draft = ast.literal_eval(f"'{draft}'")
    except (ValueError, SyntaxError):
        return pd.Series([None, None, None, None], index=['draft_rd', 'draft_overall', 'draft_yr', 'draft_team'])
    
    # Updated regex to handle ordinal numbers
    match = re.match(r'Draft: (\d+)[a-z]{2} round, (\d+)[a-z]{2} overall of the (\d{4}) NFL draft by the (.+)', draft)
    if match:
        draft_rd, draft_overall, draft_yr, draft_team = match.groups()
        return pd.Series([draft_rd, draft_overall, draft_yr, draft_team], index=['draft_rd', 'draft_overall', 'draft_yr', 'draft_team'])
    
    # Manual extraction fallback
    parts = draft.split(' ')
    if len(parts) >= 12:
        draft_rd = parts[1]
        draft_overall = ''.join(filter(str.isdigit, parts[4]))  # Extract digits only from the overall part
        draft_yr = parts[8]
        draft_team = ' '.join(parts[12:])
        return pd.Series([draft_rd, draft_overall, draft_yr, draft_team], index=['draft_rd', 'draft_overall', 'draft_yr', 'draft_team'])
    
    return pd.Series([None, None, None, None], index=['draft_rd', 'draft_overall', 'draft_yr', 'draft_team'])

# Apply function to each row
df[['draft_rd', 'draft_overall', 'draft_yr', 'draft_team']] = df['draft'].apply(split_draft_col)

display(df[['player', 'draft', 'draft_rd', 'draft_overall', 'draft_yr', 'draft_team']].head())

# Subset DataFrame where draft is not None but new columns have None values
subset_df = df[df['draft'].notna() & (
    df['draft_rd'].isna() | df['draft_overall'].isna() | df['draft_yr'].isna() | df['draft_team'].isna()
)]
display(f'rows that didn\'t get parsed into draft cols: {len(subset_df)}')
display(subset_df[['player', 'draft', 'draft_rd', 'draft_overall', 'draft_yr', 'draft_team']].head())

,player,draft,draft_rd,draft_overall,draft_yr,draft_team
0,Greg Cook,None,None,None,None,None
1,Marty Domres,None,None,None,None,None
2,Terry Hanratty,"Draft: 2nd round, 30th overall of the 1969 NFL draft by the Pittsburgh Steelers",2,30,1969,Pittsburgh Steelers
3,Bobby Douglass,"Draft: 2nd round, 41st overall of the 1969 NFL draft by the Chicago Bears",2,41,1969,Chicago Bears
4,Al Woodall,"Draft: 2nd round, 52nd overall of the 1969 NFL draft by the New York Jets",2,52,1969,New York Jets


"rows that didn't get parsed into draft cols: 0"

,player,draft,draft_rd,draft_overall,draft_yr,draft_team


In [6]:
# pd.set_option('display.max_colwidth', None)
display(df.info())
x=18
display(df.iloc[:,:x].head())
display(df.iloc[:,x:].head())
display(df.columns)
# display(df[['player','draft']]).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player         535 non-null    object
 1   G              535 non-null    object
 2   Cmp            535 non-null    object
 3   Att            535 non-null    object
 4   Cmp%           535 non-null    object
 5   Yds            535 non-null    object
 6   TD             535 non-null    object
 7   TD%            535 non-null    object
 8   Int            535 non-null    object
 9   Int%           535 non-null    object
 10  Y/A            535 non-null    object
 11  AY/A           535 non-null    object
 12  Y/C            535 non-null    object
 13  Y/G            535 non-null    object
 14  Rate           535 non-null    object
 15  blank          533 non-null    object
 16  awards         535 non-null    object
 17  school         487 non-null    object
 18  draft          491 non-null   

None

,player,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,blank,awards,school
0,Greg Cook,29,332,655,50.7,4906,34,5.2,35,5.3,7.5,6.12,14.8,169.2,120.0,,,Cincinnati
1,Marty Domres,27,368,702,52.4,4492,24,3.4,38,5.4,6.4,4.65,12.2,166.4,106.6,,,Columbia
2,Terry Hanratty,29,304,550,55.3,4152,27,4.9,34,6.2,7.5,5.75,13.7,143.2,122.5,,"H-8, H-10, H-3, AA",Notre Dame
3,Bobby Douglass,30,183,379,48.3,2817,20,5.3,20,5.3,7.4,6.11,15.4,93.9,117.6,,H-7,Kansas
4,Al Woodall,20,118,221,53.4,1501,4,1.8,13,5.9,6.8,4.51,12.7,75.1,104.7,,,Duke


,draft,pro_stats,draft_rd,draft_overall,draft_yr,draft_team
0,None,https://www.pro-football-reference.com/players/C/CookGr00.htm,None,None,None,None
1,None,https://www.pro-football-reference.com/players/D/DomrMa00.htm,None,None,None,None
2,"Draft: 2nd round, 30th overall of the 1969 NFL draft by the Pittsburgh Steelers",https://www.pro-football-reference.com/players/H/HanrTe00.htm,2,30,1969,Pittsburgh Steelers
3,"Draft: 2nd round, 41st overall of the 1969 NFL draft by the Chicago Bears",https://www.pro-football-reference.com/players/D/DougBo00.htm,2,41,1969,Chicago Bears
4,"Draft: 2nd round, 52nd overall of the 1969 NFL draft by the New York Jets",https://www.pro-football-reference.com/players/W/WoodAl00.htm,2,52,1969,New York Jets


Index(['player', 'G', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%',
       'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'blank', 'awards', 'school',
       'draft', 'pro_stats', 'draft_rd', 'draft_overall', 'draft_yr',
       'draft_team'],
      dtype='object')

In [7]:
df['awards'].value_counts()

awards
                          363
H-9                        16
H-7                        14
H-5                        13
H-10                       11
                         ... 
H-8, H-4, AA                1
AA                          1
H-9, H-4, AA                1
H-8, H-3, AA                1
H-2, Maxwell, AA, H-10      1
Name: count, Length: 66, dtype: int64

In [8]:
# df     (DataFrame)

# Export/Save as pkl
import pickle
pkl_path = "../cleaning_artifacts/cfb_ref_cleaning_output.pkl"
with open(pkl_path, 'wb') as file:
    pickle.dump(df, file)


# Export/Save as CSV 
import csv
csv_path = "../cleaning_artifacts/cfb_ref_cleaning_output.csv"
df.to_csv(csv_path, index=False)


### CELLS AFTER HERE NOT USED

## RESTORE POINT: load/restore dataframe and list from pickle

In [ ]:
# Read the PKls

# df PKl import   (DataFrame)
import pickle
pkl_path = "../cleaning_artifacts/cfb_ref_cleaning_output.pkl"
with open(pkl_path, 'rb') as f:
    restored_df = pickle.load(f)
display(f'rows : {len(restored_df)}; cols: {len(restored_df.columns)}')
display(restored_df.head())

display(f'rows : {len(restored_df)}; cols: {len(restored_df.columns)}')
display(restored_df.head())


In [ ]:
# # combined_df     (DataFrame)
# csv_path = "../cleaning_artifacts/cfb_ref_cleaning_output.csv"
# restored_df = pd.read_csv(csv_path)

# display(f'rows : {len(restored_df)}; cols: {len(restored_df.columns)}')
# display(restored_df.head())